In [1]:
import pandas as pd
from statistics import mean, stdev
import numpy as np

inFolder = "D:/Covid_data_New/Yr2023/5May/18May/"
inFile = inFolder+"Plate 160 True Mark  Psyche 19 Apr 2023_20230516 125556.xlsx"
readSepsisFile = pd.read_excel(inFile,sheet_name="Results",skiprows=23)

reporterDyesName = np.unique(readSepsisFile["Reporter"])

d:\Python\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
repIdentifier = "-"
data = dict()
clinicalCall = dict()
cqValues = dict()
tragetMapping = dict()

for i in readSepsisFile.index:
    well = readSepsisFile.loc[i,"Well"]
    wellPosition = readSepsisFile.loc[i,"Well Position"]
    sampleName = readSepsisFile.loc[i,"Sample"].split(repIdentifier)
    Cq = readSepsisFile.loc[i,"Cq"]
    reporterDye = readSepsisFile.loc[i,"Reporter"]
    targetSample = readSepsisFile.loc[i,"Target"]
    keyForDict = str(sampleName[0])+"\t"+str(targetSample)
    keyForClinicalCall = str(well)+"\t"+str(wellPosition)+"\t"+str(readSepsisFile.loc[i,"Sample"])
    if Cq == "Undetermined":
        continue
    data.setdefault(keyForDict,[]).append(Cq)
    clinicalCall.setdefault(keyForClinicalCall,[]).append(targetSample)
    cqValues.setdefault(keyForClinicalCall,[]).append(reporterDye+"\t"+str(Cq))
    tragetMapping.setdefault(keyForClinicalCall,[]).append(reporterDye)




In [71]:
meanValue = []
targetName = []
sample = []
for key in data:
    k = key.split("\t")
    avg = mean(data.get(key))
    meanValue.append(avg)
    targetName.append(k[1])
    sample.append(k[0])
dataCqMean = zip(sample,targetName,meanValue)
meanTableDf = pd.DataFrame(dataCqMean, columns=['SampleName', 'Target','CqValue_Mean'])
cqmeanTableDf=meanTableDf.pivot(index='Target', columns='SampleName', values='CqValue_Mean')
#print(cqmeanTableDf)

In [70]:
bigDF = pd.DataFrame()
header = ['Well','well Position','Sample']
header.extend(reporterDyesName)
header.append('Call')
for key in clinicalCall:
    listCqValues = []
    for dyeName in reporterDyesName:
        matchDyeNames = 0
        for cqValue in cqValues.get(key):
            dyeNameAndCq = cqValue.split("\t")
            if dyeNameAndCq[0] == dyeName:
                listCqValues.append(dyeNameAndCq[1])
                matchDyeNames=1
        if matchDyeNames == 0:
            listCqValues.append("")
    call = "|".join(clinicalCall.get(key))
   
    d = key.split("\t")
    listL=[d[0],d[1],d[2]]
    listL.extend(listCqValues)
    listL.append(call)
 
    dataFrame = pd.DataFrame([listL],[call])
    bigDF = pd.concat([bigDF,dataFrame])

completeData1 = pd.DataFrame(bigDF.values[0:],columns= header)
excelWriter = pd.ExcelWriter(inFolder+"/aa.xlsx")
cqmeanTableDf.to_excel(excel_writer=excelWriter,sheet_name= "mean")
completeData1.to_excel(excel_writer=excelWriter,sheet_name= "Data",index=False)
excelWriter.close()

['Well', 'well Position', 'Sample', 'FAM', 'JUN', 'VIC', 'Call']
